Now that we have a dataset, we can start to do some actual analysis. I'm going to be attempting to replicate the methodology of this paper:

Sapienza, Anna and Goyal, Palash and Ferrara, Emilio. Deep Neural Networks for Optimal Team Composition. Frontiers in Big Data, vol 2. Jun 2019. https://arxiv.org/abs/1805.03285 

While roller derby and esports games like League of Legends obviously are very different, in many ways, they can be treated similarly- each League match and individual jam of a derby bout consists of a team of 5 players with different defined roles attempting to achieve an objective while slowing the opposing team's attempt to achieve theirs.

A derby bout (game) consists of a series of many individual jams. Each team forwards a defensive line of four "blockers" and an offensive line of one "jammer". The jammer scores points by passing through the "pack" of blockers- one initial non-scoring pass through the pack is required, and then one point is earned for each of the opposing team's blockers that the jammer passes on subsequent laps. Each jam can run for a set amount of time, but the jammer that is the first to complete the non-scoring pass ("lead jammer") can choose to end the jam early. In addition, the jammer can hand off their jammer status to one special blocker on each team called a "pivot" by passing the special helmet cover that the jammer wears. This is the general gist of the sport- in many ways, it's similar to the playground game "Red Rover", but on wheels.

Naturally, when the blockers try to stop the jammer, things can get scrappy! Various penalties are given when a player shoves another in an illegal manner, when a blocker strays too far from the pack, when a player goes out of bounds, when a blocker makes an illegal formation (such as linking arms with another blocker), etc. It's general "derby wisdom" that certain penalties are more common "new-skater" penalties, while the distribution of penalties changes with skill. We can test this!


Let's pick a team. I'll use the Kalamazoo Derby Darlins, the team I've announced for for the past few years. 

In this analysis, I'm going to make some assumptions.
-First, that the fundamental unit of derby is not the bout, but the jam. Each jam is unique, and may have starting conditions determined by the preceding jam, but ultimately, for the purposes of this analysis, the only influence jam 1 may have on a jam like jam 20 is player stamina (N.B.: sometimes players can still be in the penalty box from previous jams, so this is not strictly correct! but it's probably correct enough for what we'd like to test here). This means that I will update a player's "rating" each jam rather than each bout.

-Second, that the "figure of merit" to determine the performance of a jammer is the total number of points they score in a jam, but that the "figure of merit" to determine the performance of a blocker line is the difference between their jammer's score and the opposing jammer's score. A good blocker line is able to slow the opposing jammer substantially while also letting their own through.

-Third: the rules of roller derby change often, as the sport is still relatively new. For instance- at one point, jammers scored an additional point for passing the opposing team's jammer as well as blockers.
    

In [1]:
import requests
import pandas as pd
import numpy as np
import trueskill
from bs4 import BeautifulSoup
from itertools import product
from urllib.request import urlopen
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph 
import matplotlib.pyplot as plt
import pylab

import nbimporter
import Webscraper as wsc


teamID=str(3637)
teamName='Killamazoo'

In [2]:
#First, get the lineups for each jam KDD has stats available for.
AllLineups = wsc.GetAllLineups(teamID, teamName)

# Also, get expanding average of score differentials for each jam. We'll use a player's
# average score differential after a given jam as a proxy for their skill ranking as measured
# after playing that jam.

AllAvgs = wsc.ExpandingAverages(teamID, teamName)
badjams,badblockers = wsc.GetBadJamsAndBlockers(teamID, teamName,12)
print(badjams)

Webscraper.ipynb:7: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 7 of the file Webscraper.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  "In this series of notebooks, I will attempt to do some introductory exploration of various roller derby statistics. We will use the publicly available stats on the FlatTrackStats website. First, I will build a table scraper tool using the BeautifulSoup4 package to parse the stats tables on the website. If not already installed, you will need pandas and BeautifulSoup4 in order to run this notebook. "
Webscraper.ipynb:16: GuessedAtParserWarning: No parser was explicitly specified, so I'm u

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 39, 41, 42, 45, 47, 49, 54, 56, 59, 60, 64, 68, 72, 80, 84, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 115, 116, 117, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 160, 161, 163, 167, 168, 169, 171, 172, 173, 174, 175, 176, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 191, 192, 193, 195, 199, 212, 214, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 228, 230, 232, 234, 236, 261, 262, 296, 302, 306, 355, 357, 359, 361, 363, 364, 365, 366, 369, 373, 374, 375, 395, 397, 398, 399, 403, 405, 407, 409, 411, 414, 415, 417, 418, 419, 420, 421, 422, 424, 425, 427, 428, 430, 432, 434, 435, 440, 442, 444, 450, 452, 456, 457, 458

Let's only look at blockers for now, since they interact most closely with each other. Matching jammers to blocker lines is a different question than composing the lines themselves, since interplay is different.

In [3]:
print(AllLineups, AllAvgs)

              Jammer     Jstats              B1              B2  \
0         Beaver Jam      Lead   Aly-Kate Co...  Wreck Keene...   
1       Buns N Roses          0  Painbow Con...  Smash Bandi...   
2        Weers Waldo          0  Painbow Con...  Aly-Kate Co...   
3      Hill-De-Beast          0  Smash Bandi...   Beverly Hells   
4         Beaver Jam          0  Smash Bandi...  Wreck Keene...   
...              ...        ...             ...             ...   
1063   Beverly Hells          0  Sparkills (...  Rosie Feroc...   
1064  Delilah Danger      Lead          Javelin  Ivanna O'Bl...   
1065  Rosie Feroc...          0       Lady Hawk   Noam Stompsky   
1066  Sparkills (...  LeadLoss   Ivanna O'Bl...  Ramona D. F...   
1067   Beverly Hells          0         Javelin  Delilah Danger   

                  B3              B4  jamscore  runscore  ScoreDiff  
0            Mustang         Javelin        10        10       10.0  
1     Maggie Walters  Ophelia Plenty         0        1

Next, let's build the short term play network described in the paper. 

In [19]:
blockerlines = AllLineups[['B1', 'B2', 'B3', 'B4']]
#print(blockerlines)

STjams=[]
for jamnum in range(len((blockerlines.index))):
    
    if (jamnum in badjams): continue
    G = nx.complete_graph(4, nx.DiGraph())
    blockers = blockerlines.iloc[jamnum].to_list()
    mapping = dict(zip(G, blockers))
    G = nx.relabel_nodes(G, mapping)
    
    for edge in G.edges():
        weight = AllAvgs.iloc[jamnum][edge[0]]-AllAvgs.iloc[jamnum-1][edge[0]]
        #print(weight)
        G[edge[0]][edge[1]]['weight'] = weight
        STjams.append(G)

{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.4250000000000007}
{'weight': 0.4250000000000007}
{'weight': 0.4250000000000007}
{'weight': 0.3516483516483522}
{'weight': 0.3516483516483522}
{'weight': 0.3516483516483522}
{'weight': 0.07916666666666661}
{'weight': 0.07916666666666661}
{'weight': 0.07916666666666661}
{'weight': 0.335164835164834}
{'weight': 0.335164835164834}
{'weight': 0.335164835164834}
{'weight': -0.23976608187134474}
{'weight': -0.23976608187134474}
{'weight': -0.23976608187134474}
{'weight': 0.022058823529411242}
{'weight': 0.022058823529411242}
{'weight': 0.022058823529411242}
{'weight': -0.0952380952380949}
{'weight': -0.0952380952380949}
{'weight': -0.0952380952380949}
{'weight': -0.28308823529411775}
{'weight': -0.28308823529411775}
{'weight': -0.28308823529411775}
{'weight': 0.1703296703296706}
{'weight': 0

{'weight': 0.32183908045977017}
{'weight': 0.32183908045977017}
{'weight': 0.32183908045977017}
{'weight': 0.18538324420677377}
{'weight': 0.18538324420677377}
{'weight': 0.18538324420677377}
{'weight': 0.7169230769230772}
{'weight': 0.7169230769230772}
{'weight': 0.7169230769230772}
{'weight': 0.48873873873873874}
{'weight': 0.48873873873873874}
{'weight': 0.48873873873873874}
{'weight': 0.645320197044335}
{'weight': 0.645320197044335}
{'weight': 0.645320197044335}
{'weight': 1.2121212121212122}
{'weight': 1.2121212121212122}
{'weight': 1.2121212121212122}
{'weight': -0.23286290322580605}
{'weight': -0.23286290322580605}
{'weight': -0.23286290322580605}
{'weight': -0.1904761904761898}
{'weight': -0.1904761904761898}
{'weight': -0.1904761904761898}
{'weight': -0.6066176470588234}
{'weight': -0.6066176470588234}
{'weight': -0.6066176470588234}
{'weight': -0.20564516129032295}
{'weight': -0.20564516129032295}
{'weight': -0.20564516129032295}
{'weight': -0.40056818181818166}
{'weight': -0

{'weight': 0.03968253968253954}
{'weight': 0.025846702317290582}
{'weight': 0.025846702317290582}
{'weight': 0.025846702317290582}
{'weight': 0.028605015673980994}
{'weight': 0.028605015673980994}
{'weight': 0.028605015673980994}
{'weight': 0.04878048780487809}
{'weight': 0.04878048780487809}
{'weight': 0.04878048780487809}
{'weight': 0.07142857142857162}
{'weight': 0.07142857142857162}
{'weight': 0.07142857142857162}
{'weight': 0.05043411644535256}
{'weight': 0.05043411644535256}
{'weight': 0.05043411644535256}
{'weight': 0.09302325581395365}
{'weight': 0.09302325581395365}
{'weight': 0.09302325581395365}
{'weight': 0.42727272727272725}
{'weight': 0.42727272727272725}
{'weight': 0.42727272727272725}
{'weight': 0.1333333333333333}
{'weight': 0.1333333333333333}
{'weight': 0.1333333333333333}
{'weight': 0.13424947145877342}
{'weight': 0.13424947145877342}
{'weight': 0.13424947145877342}
{'weight': 0.0715355805243445}
{'weight': 0.0715355805243445}
{'weight': 0.0715355805243445}
{'weight

{'weight': 0.009512937595129278}
{'weight': 0.009512937595129278}
{'weight': 0.2847658979734451}
{'weight': 0.2847658979734451}
{'weight': 0.2847658979734451}
{'weight': 0.14602784602784613}
{'weight': 0.14602784602784613}
{'weight': 0.14602784602784613}
{'weight': 0.24654377880184342}
{'weight': 0.24654377880184342}
{'weight': 0.24654377880184342}
{'weight': 0.3256038647342996}
{'weight': 0.3256038647342996}
{'weight': 0.3256038647342996}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.09159663865546218}
{'weight': 0.09159663865546218}
{'weight': 0.09159663865546218}
{'weight': 0.12026515151515149}
{'weight': 0.12026515151515149}
{'weight': 0

{'weight': 0.1914160401002506}
{'weight': 0.1914160401002506}
{'weight': 0.1914160401002506}
{'weight': 0.12434210526315792}
{'weight': 0.12434210526315792}
{'weight': 0.12434210526315792}
{'weight': -0.015868351454598972}
{'weight': -0.015868351454598972}
{'weight': -0.015868351454598972}
{'weight': -0.00584376863446634}
{'weight': -0.00584376863446634}
{'weight': -0.00584376863446634}
{'weight': -0.025141242937853026}
{'weight': -0.025141242937853026}
{'weight': -0.025141242937853026}
{'weight': -0.030985915492957705}
{'weight': -0.030985915492957705}
{'weight': -0.030985915492957705}
{'weight': 0.22360248447204967}
{'weight': 0.22360248447204967}
{'weight': 0.22360248447204967}
{'weight': 0.25378100423472477}
{'weight': 0.25378100423472477}
{'weight': 0.25378100423472477}
{'weight': 0.16301546391752586}
{'weight': 0.16301546391752586}
{'weight': 0.16301546391752586}
{'weight': 0.20598591549295775}
{'weight': 0.20598591549295775}
{'weight': 0.20598591549295775}
{'weight': 0.0}
{'weig

...

{'weight': -0.013332514131236178}
{'weight': -0.7307692307692308}
{'weight': -0.7307692307692308}
{'weight': -0.7307692307692308}
{'weight': -0.025142662049456366}
{'weight': -0.025142662049456366}
{'weight': -0.025142662049456366}
{'weight': -0.05669963419590829}
{'weight': -0.05669963419590829}
{'weight': -0.05669963419590829}
{'weight': 0.0058606865957143706}
{'weight': 0.0058606865957143706}
{'weight': 0.0058606865957143706}
{'weight': -0.2333333333333334}
{'weight': -0.2333333333333334}
{'weight': -0.2333333333333334}
{'weight': 0.19523809523809532}
{'weight': 0.19523809523809532}
{'weight': 0.19523809523809532}
{'weight': -0.019813519813519864}
{'weight': -0.019813519813519864}
{'weight': -0.019813519813519864}
{'weight': 0.10000000000000009}
{'weight': 0.10000000000000009}
{'weight': 0.10000000000000009}
{'weight': -0.01264591439688717}
{'weight': -0.01264591439688717}
{'weight': -0.01264591439688717}
{'weight': -0.010049539985845707}
{'weight': -0.010049539985845707}
{'weight':

{'weight': 0.0806878306878307}
{'weight': 0.011105327086586847}
{'weight': 0.011105327086586847}
{'weight': 0.011105327086586847}
{'weight': 0.06768523744255422}
{'weight': 0.06768523744255422}
{'weight': 0.06768523744255422}
{'weight': 0.3598790322580645}
{'weight': 0.3598790322580645}
{'weight': 0.3598790322580645}
{'weight': 0.4836601307189543}
{'weight': 0.4836601307189543}
{'weight': 0.4836601307189543}
{'weight': 0.08211473565804273}
{'weight': 0.08211473565804273}
{'weight': 0.08211473565804273}
{'weight': 0.01409749670619237}
{'weight': 0.01409749670619237}
{'weight': 0.01409749670619237}
{'weight': 0.38157894736842124}
{'weight': 0.38157894736842124}
{'weight': 0.38157894736842124}
{'weight': 0.06432748538011701}
{'weight': 0.06432748538011701}
{'weight': 0.06432748538011701}
{'weight': 0.011040193203381066}
{'weight': 0.011040193203381066}
{'weight': 0.011040193203381066}
{'weight': 0.06534090909090909}
{'weight': 0.06534090909090909}
{'weight': 0.06534090909090909}
{'weight'

{'weight': 0.30219780219780223}
{'weight': 0.30219780219780223}
{'weight': 0.27916666666666656}
{'weight': 0.27916666666666656}
{'weight': 0.27916666666666656}
{'weight': 0.04260651629072687}
{'weight': 0.04260651629072687}
{'weight': 0.04260651629072687}
{'weight': 0.05204872646733116}
{'weight': 0.05204872646733116}
{'weight': 0.05204872646733116}
{'weight': 0.007510003077870142}
{'weight': 0.007510003077870142}
{'weight': 0.007510003077870142}
{'weight': 1.2916666666666667}
{'weight': 1.2916666666666667}
{'weight': 1.2916666666666667}
{'weight': 0.3169993950393224}
{'weight': 0.3169993950393224}
{'weight': 0.3169993950393224}
{'weight': 0.11769920220624419}
{'weight': 0.11769920220624419}
{'weight': 0.11769920220624419}
{'weight': 0.1054082785196997}
{'weight': 0.1054082785196997}
{'weight': 0.1054082785196997}
{'weight': 0.018842268842268695}
{'weight': 0.018842268842268695}
{'weight': 0.018842268842268695}
{'weight': 0.09513742071881603}
{'weight': 0.09513742071881603}
{'weight': 

{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.024011299435028333}
{'weight': 0.024011299435028333}
{'weight': 0.024011299435028333}
{'weight': 0.03076923076923077}
{'weight': 0.03076923076923077}
{'weight': 0.03076923076923077}
{'weight': 0.00799120966936362}
{'weight': 0.00799120966936362}
{'weight': 0.00799120966936362}
{'weight': 0.583333333333333}
{'weight': 0.583333333333333}
{'weight'

{'weight': -0.2740863787375416}
{'weight': -0.02637352650984054}
{'weight': -0.02637352650984054}
{'weight': -0.02637352650984054}
{'weight': -0.08730158730158744}
{'weight': -0.08730158730158744}
{'weight': -0.08730158730158744}
{'weight': -0.09618794326241137}
{'weight': -0.09618794326241137}
{'weight': -0.09618794326241137}
{'weight': -0.03724340175953078}
{'weight': -0.03724340175953078}
{'weight': -0.03724340175953078}
{'weight': -0.013377883441495875}
{'weight': -0.013377883441495875}
{'weight': -0.013377883441495875}
{'weight': -0.2759353741496598}
{'weight': -0.2759353741496598}
{'weight': -0.2759353741496598}
{'weight': -0.3916256157635467}
{'weight': -0.3916256157635467}
{'weight': -0.3916256157635467}
{'weight': -0.3534482758620694}
{'weight': -0.3534482758620694}
{'weight': -0.3534482758620694}
{'weight': -0.03603759110088223}
{'weight': -0.03603759110088223}
{'weight': -0.03603759110088223}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'weight': 0.0}
{'w

In [42]:
STGraph = nx.DiGraph()
for jam in STjams:
    for edge in jam.edges():
        if STGraph.has_edge(*edge):
            weightsum = jam.get_edge_data(*edge)['weight'] + STGraph.get_edge_data(*edge)['weight'] 
            STGraph[edge[0]][edge[1]]['weight'] = weightsum
        else: 
            #print("no edge yet")
            STGraph.add_edge(*edge[:2])
            STGraph[edge[0]][edge[1]]['weight'] = 0

nx.write_weighted_edgelist(STGraph, "Data/"+teamID+"STGraph.edgelist")

[('Wreck Keene Ball', 'Ophelia Plenty', {'weight': 7.324979561501303}),
 ('Wreck Keene Ball', 'Maggie Walters', {'weight': 0.7719298245614041}),
 ('Wreck Keene Ball', 'Smash Bandicute', {'weight': 12.222009569377988}),
 ('Wreck Keene Ball', 'Mea Chokeavich', {'weight': 22.18646616541352}),
 ('Wreck Keene Ball', 'Javelin', {'weight': 16.898131809710762}),
 ('Wreck Keene Ball', 'Aly-Kate Contusion...', {'weight': 27.563339955239265}),
 ('Ophelia Plenty', 'Wreck Keene Ball', {'weight': 10.22603744082004}),
 ('Ophelia Plenty', 'Maggie Walters', {'weight': 1.590640112725863}),
 ('Ophelia Plenty', 'Smash Bandicute', {'weight': 8.730249628337724}),
 ('Ophelia Plenty', 'Aly-Kate Contusion...', {'weight': 21.20234113712371}),
 ('Ophelia Plenty', 'Beverly Hells', {'weight': -6.800387034278502}),
 ('Ophelia Plenty', 'Mea Chokeavich', {'weight': 11.080464768206703}),
 ('Ophelia Plenty', 'Javelin', {'weight': 22.602317504842013}),
 ('Ophelia Plenty', 'Protasiewicz', {'weight': -4.297116092584525}),

In [ ]:
#get nodes and edges from the STGraph, remove weights
LTGraph = STGraph

for n1, n2, d in LTGraph.edges(data=True):
    d.pop('weight', None) 
    
for edge in LTGraph.edges():
     LTGraph[edge[0]][edge[1]]['jamssince'] = 0
        
        
#Add a new edge feature: "jams since last co-play" that updates each jam, and use it to get the weights    

for jamnum in range(len((blockerlines.index))):
    #get all edges in jam
    G = nx.complete_graph(4, nx.DiGraph())
    blockers = blockerlines.iloc[jamnum].to_list()
    mapping = dict(zip(G, blockers))
    G = nx.relabel_nodes(G, mapping)
    
    #get all possible combos
    for edge in LTGraph.edges():
        #zero if they play together in this jam, increment otherwise
        if edge in G.edges(): LTGraph[edge[0]][edge[1]]['jamssince'] = 0
        else: LTGraph[edge[0]][edge[1]]['jamssince'] += 1     

    if (jamnum in badjams): continue
    
    
     
    
    
    # Get all blockers in the jam
    # Get all possible teammates
       G.out_edges(node) 
    # update 
    
    
    
    
    
    
    
    
    




for jamnum in range(len((blockerlines.index))):
    G = nx.complete_graph(4, nx.DiGraph())
    blockers = blockerlines.iloc[jamnum].to_list()
    mapping = dict(zip(G, blockers))
    G = nx.relabel_nodes(G, mapping)
    for edge in G.edges():
        last
    
    
    
    
    
    
    
    
    
    
    
    
    
    if (jamnum in badjams): continue    
    for edge in G.edges():
        weight = AllAvgs.iloc[jamnum][edge[0]]-AllAvgs.iloc[jamnum-1][edge[0]]
        #print(weight)
        G[edge[0]][edge[1]]['weight'] = weight
        STjams.append(G)
